In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Función para recomendar películas
import numpy as np

# Configuración para optimizar el uso de la CPU
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)

# Cargar el archivo CSV
file_path = 'movies.csv'
movies_df = pd.read_csv(file_path)

# Convertir la columna 'genres' de strings a listas
movies_df['genres'] = movies_df['genres'].apply(eval)

# Usar MultiLabelBinarizer para convertir los géneros a una representación one-hot
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies_df['genres'])

# Crear un nuevo DataFrame con los géneros codificados
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)

# Concatenar los géneros codificados con el DataFrame original
movies_df = pd.concat([movies_df, genres_df], axis=1)

# Eliminar las columnas que no serán usadas
movies_df.drop(['release_date'], axis=1, inplace=True)

# Separar características y etiquetas
X = movies_df.drop('movie_id', axis=1).values
y = movies_df.drop('movie_id', axis=1).values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de red neuronal
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')



def recommend_movies(model, input_genres, num_recommendations=5):
    # Convertir los géneros de entrada a una representación one-hot
    input_vector = mlb.transform([input_genres])
    
    # Predecir las probabilidades de géneros con el modelo entrenado
    genre_probabilities = model.predict(input_vector)[0]
    
    # Seleccionar los géneros con las probabilidades más altas
    recommended_genres = np.argsort(-genre_probabilities)[:num_recommendations]
    
    # Mapear los índices a los nombres de los géneros
    recommended_genres_names = mlb.classes_[recommended_genres]
    
    return recommended_genres_names

# Ejemplo de uso
input_genres = ['Accion', 'Aventura']
recommendations = recommend_movies(model, input_genres)
print('Recomendaciones de géneros:', recommendations)


/Users/nono/Library/Python/3.9/lib/python/site-packages/sklearn/utils/__init__.py:20: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.19.5)
  from scipy.sparse import issparse


ImportError: cannot import name '__all__' from 'numpy.linalg' (/Users/nono/Library/Python/3.9/lib/python/site-packages/numpy/linalg/__init__.py)